In [23]:
import pandas as pd
import numpy as np
import os

import statsmodels.formula.api as smf
from statsmodels.formula.api import ols

from scipy.stats import shapiro, ttest_ind, mannwhitneyu

In [24]:
base = os.path.dirname(os.getcwd())
stats_path = os.path.join(base, 'csv files', 'For_Stats.csv')

stats_df = pd.read_csv(stats_path)

In [25]:
same = stats_df[stats_df['is_same'] == 1]['judge_goe']
not_same = stats_df[stats_df['is_same'] == 0]['judge_goe']

In [26]:
print(f"All scores: {len(stats_df)}\n"
      f"Scores equal to panel median: {len(stats_df[stats_df['goe_dist'] == 0])}\n"
      f"Scores higher or lower than panel median: {len(stats_df[stats_df['goe_dist'] != 0])}")

All scores: 209619
Scores equal to panel median: 124272
Scores higher or lower than panel median: 85347


In [27]:
base = os.path.dirname(os.getcwd())
stats_path = os.path.join(base, 'csv files', 'For_Stats.csv')

stats_df = pd.read_csv(stats_path)

In [28]:
same = stats_df[stats_df['is_same'] == 1]['judge_goe']
not_same = stats_df[stats_df['is_same'] == 0]['judge_goe']

In [29]:
print(f"All scores: {len(stats_df)}\n"
      f"Scores equal to the panel median: {len(stats_df[stats_df['goe_dist'] == 0])}\n"
      f"Scores higher or lower than the panel median: {len(stats_df[stats_df['goe_dist'] != 0])}")

All scores: 209619
Scores equal to the panel median: 124272
Scores higher or lower than the panel median: 85347


In [30]:
nx, ny = len(same), len(not_same)
print(f"Sample sizes: {nx}, {ny}")

Sample sizes: 16088, 193531


In [31]:
sample_0 = same.sample(5000, random_state = 42)
sample_1 = not_same.sample(5000, random_state = 42)

In [32]:
stat_0, p_0 = shapiro(sample_0)
stat_1, p_1 = shapiro(sample_1)

In [33]:
print(f"Group is_same=0: p={p_0:.5f}, {'Normal Distribution' if p_0 > 0.05 else 'Not Normal Distribution'}")
print(f"Group is_same=1: p={p_1:.5f}, {'Normal Distribution' if p_0 > 0.05 else 'Not Normal Distribution'}")

Group is_same=0: p=0.00000, Not Normal Distribution
Group is_same=1: p=0.00000, Not Normal Distribution


In [34]:
U1, p = mannwhitneyu(same, not_same, method='asymptotic', alternative="two-sided")
U2 = nx * ny - U1

In [35]:
print(f"\nMann-Whitney U-Test: U1_stat={U1:.4f}, p={p}")
print(f"\nMann-Whitney U-Test: U2_stat={U2:.4f}")


Mann-Whitney U-Test: U1_stat=1706749269.0000, p=7.594038389084582e-95

Mann-Whitney U-Test: U2_stat=1406777459.0000


In [36]:
mi_U1 = nx * ny / 2
sigma_U1 = np.sqrt(nx * ny * (nx + ny + 1) / 12)
Z = (U1 - mi_U1) / sigma_U1

r = Z / np.sqrt(nx + ny)

print(r)

0.04442046403786992


In [37]:
stats_df

,rank,name,nation,startnr,total,tech,pcs,deductions,competition,element,...,panel_median,judge_id,judge_goe,judge_nation,is_same,judge_name,goe_dist,higher,lower,pt_bias
0,1,Michal BREZINA,CZE,26,89.77,48.05,41.72,0.0,ec2020SEG001OF,4S+2T,...,2.0,Judge No.1,1.0,GEO,0,Salome CHIGOGIDZE,1.0,0,1,0
1,1,Michal BREZINA,CZE,26,89.77,48.05,41.72,0.0,ec2020SEG001OF,3F,...,3.0,Judge No.1,2.0,GEO,0,Salome CHIGOGIDZE,1.0,0,1,0
2,1,Michal BREZINA,CZE,26,89.77,48.05,41.72,0.0,ec2020SEG001OF,FSSp4,...,3.0,Judge No.1,3.0,GEO,0,Salome CHIGOGIDZE,0.0,0,0,0
3,1,Michal BREZINA,CZE,26,89.77,48.05,41.72,0.0,ec2020SEG001OF,3A,...,3.0,Judge No.1,3.0,GEO,0,Salome CHIGOGIDZE,0.0,0,0,0
4,1,Michal BREZINA,CZE,26,89.77,48.05,41.72,0.0,ec2020SEG001OF,StSq4,...,4.0,Judge No.1,2.0,GEO,0,Salome CHIGOGIDZE,2.0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209614,24,Meda VARIAKOJYTE,LTU,1,88.83,48.22,40.61,0.0,wc2025SEG004OF,2A+2T,...,0.0,Judge No.9,0.0,FRA,0,Florence VUYLSTEKER,0.0,0,0,0
209615,24,Meda VARIAKOJYTE,LTU,1,88.83,48.22,40.61,0.0,wc2025SEG004OF,3Sq+2T,...,-5.0,Judge No.9,-5.0,FRA,0,Florence VUYLSTEKER,0.0,0,0,0
209616,24,Meda VARIAKOJYTE,LTU,1,88.83,48.22,40.61,0.0,wc2025SEG004OF,LSp2,...,1.0,Judge No.9,-1.0,FRA,0,Florence VUYLSTEKER,2.0,0,1,0
209617,24,Meda VARIAKOJYTE,LTU,1,88.83,48.22,40.61,0.0,wc2025SEG004OF,StSq3,...,0.0,Judge No.9,-2.0,FRA,0,Florence VUYLSTEKER,2.0,0,1,0


In [38]:
all_country_combinations = []

for sc in stats_df['nation'].unique():
    for jc in stats_df['judge_nation'].unique():
        all_country_combinations.append((sc, jc))

len(all_country_combinations), len(set(all_country_combinations))

(2744, 2744)

In [61]:
skater_judge_com = stats_df.apply(lambda row: row['nation'] + row['judge_nation'], axis = 1)
print(len(skater_judge_com.unique()))
print(skater_judge_com.value_counts().head(44))

1819
JPNUSA    3131
JPNJPN    3092
JPNCAN    2684
USAUSA    2334
JPNKOR    2309
USAJPN    2237
USACAN    2080
JPNFRA    1803
USAKOR    1641
CANCAN    1469
JPNITA    1466
CANUSA    1308
KORJPN    1266
CANJPN    1247
USAFRA    1221
JPNFIN    1211
KORUSA    1194
KORKOR    1192
RUSRUS    1152
JPNCHN    1151
KORCAN    1148
JPNEST    1081
JPNGER    1043
FRAFRA    1040
USAITA    1031
RUSCAN    1017
RUSJPN    1002
JPNBEL     981
RUSUSA     981
JPNRUS     934
JPNSUI     908
CANKOR     855
FRAJPN     813
USAEST     808
JPNAUS     797
FRAUSA     792
JPNCZE     778
CHNJPN     763
CHNCHN     744
USAFIN     741
USACHN     737
USARUS     720
ITAITA     711
USAGER     702
Name: count, dtype: int64


In [40]:
stats_df['skater_judge'] = stats_df.apply(lambda row: row['nation'] + row['judge_nation'], axis = 1)

In [73]:
keep = set(stats_df['skater_judge'].value_counts().index[:44])
stats_trim = stats_df[stats_df['skater_judge'].isin(keep)]
stats_trim = stats_trim[stats_trim['judge_goe'] > 1.]

In [76]:
judge_counts = stats_trim['judge_name'].value_counts()
stats_trim = stats_trim[stats_trim['judge_name'].isin(judge_counts.index[:109])]

stats_trim

,rank,name,nation,startnr,total,tech,pcs,deductions,competition,element,...,judge_id,judge_goe,judge_nation,is_same,judge_name,goe_dist,higher,lower,pt_bias,skater_judge
2404,1,Kao MIURA,JPN,19,91.90,51.10,40.80,0.0,fc2023SEG001OF,3A,...,Judge No.1,3.0,KOR,0,Na Young AHN,0.0,0,0,0,JPNKOR
2405,1,Kao MIURA,JPN,19,91.90,51.10,40.80,0.0,fc2023SEG001OF,FCSp4,...,Judge No.1,2.0,KOR,0,Na Young AHN,0.0,0,0,0,JPNKOR
2406,1,Kao MIURA,JPN,19,91.90,51.10,40.80,0.0,fc2023SEG001OF,4T+3T,...,Judge No.1,4.0,KOR,0,Na Young AHN,-1.0,1,0,0,JPNKOR
2407,1,Kao MIURA,JPN,19,91.90,51.10,40.80,0.0,fc2023SEG001OF,CSSp4,...,Judge No.1,3.0,KOR,0,Na Young AHN,-1.0,1,0,0,JPNKOR
2408,1,Kao MIURA,JPN,19,91.90,51.10,40.80,0.0,fc2023SEG001OF,StSq3,...,Judge No.1,3.0,KOR,0,Na Young AHN,0.0,0,0,0,JPNKOR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209487,14,Lorine SCHILD,FRA,10,117.31,63.42,53.89,0.0,wc2025SEG004OF,3Lz+3T,...,Judge No.9,2.0,FRA,1,Florence VUYLSTEKER,0.0,0,0,0,FRAFRA
209493,14,Lorine SCHILD,FRA,10,117.31,63.42,53.89,0.0,wc2025SEG004OF,StSq3,...,Judge No.9,2.0,FRA,1,Florence VUYLSTEKER,-2.0,1,0,0,FRAFRA
209495,14,Lorine SCHILD,FRA,10,117.31,63.42,53.89,0.0,wc2025SEG004OF,ChSq1,...,Judge No.9,3.0,FRA,1,Florence VUYLSTEKER,-1.0,1,0,0,FRAFRA
209496,14,Lorine SCHILD,FRA,10,117.31,63.42,53.89,0.0,wc2025SEG004OF,3S,...,Judge No.9,2.0,FRA,1,Florence VUYLSTEKER,-1.0,1,0,0,FRAFRA


In [77]:
stats_trim['uid'] = stats_trim.apply(lambda row: row['element'] + row['name'] + row['competition'], axis=1)
stats_trim

,rank,name,nation,startnr,total,tech,pcs,deductions,competition,element,...,judge_goe,judge_nation,is_same,judge_name,goe_dist,higher,lower,pt_bias,skater_judge,uid
2404,1,Kao MIURA,JPN,19,91.90,51.10,40.80,0.0,fc2023SEG001OF,3A,...,3.0,KOR,0,Na Young AHN,0.0,0,0,0,JPNKOR,3AKao MIURAfc2023SEG001OF
2405,1,Kao MIURA,JPN,19,91.90,51.10,40.80,0.0,fc2023SEG001OF,FCSp4,...,2.0,KOR,0,Na Young AHN,0.0,0,0,0,JPNKOR,FCSp4Kao MIURAfc2023SEG001OF
2406,1,Kao MIURA,JPN,19,91.90,51.10,40.80,0.0,fc2023SEG001OF,4T+3T,...,4.0,KOR,0,Na Young AHN,-1.0,1,0,0,JPNKOR,4T+3TKao MIURAfc2023SEG001OF
2407,1,Kao MIURA,JPN,19,91.90,51.10,40.80,0.0,fc2023SEG001OF,CSSp4,...,3.0,KOR,0,Na Young AHN,-1.0,1,0,0,JPNKOR,CSSp4Kao MIURAfc2023SEG001OF
2408,1,Kao MIURA,JPN,19,91.90,51.10,40.80,0.0,fc2023SEG001OF,StSq3,...,3.0,KOR,0,Na Young AHN,0.0,0,0,0,JPNKOR,StSq3Kao MIURAfc2023SEG001OF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209487,14,Lorine SCHILD,FRA,10,117.31,63.42,53.89,0.0,wc2025SEG004OF,3Lz+3T,...,2.0,FRA,1,Florence VUYLSTEKER,0.0,0,0,0,FRAFRA,3Lz+3TLorine SCHILDwc2025SEG004OF
209493,14,Lorine SCHILD,FRA,10,117.31,63.42,53.89,0.0,wc2025SEG004OF,StSq3,...,2.0,FRA,1,Florence VUYLSTEKER,-2.0,1,0,0,FRAFRA,StSq3Lorine SCHILDwc2025SEG004OF
209495,14,Lorine SCHILD,FRA,10,117.31,63.42,53.89,0.0,wc2025SEG004OF,ChSq1,...,3.0,FRA,1,Florence VUYLSTEKER,-1.0,1,0,0,FRAFRA,ChSq1Lorine SCHILDwc2025SEG004OF
209496,14,Lorine SCHILD,FRA,10,117.31,63.42,53.89,0.0,wc2025SEG004OF,3S,...,2.0,FRA,1,Florence VUYLSTEKER,-1.0,1,0,0,FRAFRA,3SLorine SCHILDwc2025SEG004OF


In [78]:
quality = ols('judge_goe ~ C(uid, Treatment) + C(judge_name, Treatment)', data=stats_trim).fit()
quality.summary()

MemoryError: Unable to allocate 1.96 GiB for an array with shape (8811, 29920) and data type float64

In [64]:
model = smf.ols("judge_goe ~ is_same + prestige + s_progression + panel_median", data=stats_df).fit()

In [65]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:              judge_goe   R-squared:                       0.894
Model:                            OLS   Adj. R-squared:                  0.894
Method:                 Least Squares   F-statistic:                 4.420e+05
Date:                pt., 25 kwi 2025   Prob (F-statistic):               0.00
Time:                        15:50:32   Log-Likelihood:            -2.3768e+05
No. Observations:              209619   AIC:                         4.754e+05
Df Residuals:                  209614   BIC:                         4.754e+05
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        -0.0193      0.006     -3.319

In [80]:
from statsmodels.formula.api import mixedlm
model = mixedlm("judge_goe ~ 1", data=stats_trim, groups=stats_trim["judge_name"])
result = model.fit()
print(result.summary())


          Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: judge_goe  
No. Observations: 29920   Method:             REML       
No. Groups:       109     Scale:              0.5655     
Min. group size:  100     Log-Likelihood:     -34065.4657
Max. group size:  1047    Converged:          Yes        
Mean group size:  274.5                                  
----------------------------------------------------------
           Coef.  Std.Err.     z     P>|z|  [0.025  0.975]
----------------------------------------------------------
Intercept  2.694     0.017  161.471  0.000   2.661   2.727
Group Var  0.028     0.006                                

